In [11]:
from typing import List
import pandas as pd
import json
import os

def compute_validation_effectiveness(baseline_file: str, model_names: List, result_dir: str, index_name: str):

    df_baseline = pd.read_csv(baseline_file)
    
    results = []

    for model_name in model_names:
        
        results_file = f"{result_dir}/all_dependencies_{index_name}_{model_name}.json"

        with open(results_file, "r", encoding="utf-8") as src:
            data = json.load(src)

        true_positives = []
        true_negatives = []
        false_positives = []
        false_negatives = []
        accuracy_count = []

        for entry, (index, row) in zip(data, df_baseline.iterrows()):

            rating = row["final_rating"]
            try:
                if "responses" in entry:
                    response = entry["responses"][0]
                else:
                    response = entry["response"]
                response_dict = json.loads(response)
                isDependency = response_dict["isDependency"]
            except json.JSONDecodeError as error:
                #print(index)
                #print(response_dict)
                #print(type(response_dict))
                try:
                    isDependency = response_dict["isDependency"]
                except KeyError as error:
                    print(error)
                    continue
            except KeyError as error:
                print(error)
                continue
            
            if str(rating) == "Borderline":
                if isDependency:
                    accuracy_count.append(1)
                    true_positives.append(1)
                if not isDependency:
                    accuracy_count.append(1)
                    true_negatives.append(1)

            # TP: The LLM validates a dependency as correct and the dependency is correct
            if isDependency and str(rating).lower() == "true":
                accuracy_count.append(1)
                true_positives.append(1)
                
            # FP: The LLM validates a dependency as correct, but the dependency is actually incorrect
            if isDependency and str(rating).lower() == "false":
                accuracy_count.append(0)
                false_positives.append(1)

            # TN: The LLM validates a dependency as incorrect and the dependency is incorrect
            if not isDependency and str(rating).lower() == "false":
                accuracy_count.append(1)
                true_negatives.append(1)

            # FN: The LLM validates a dependency as incorrect, but the dependency is actually correct
            if not isDependency and  str(rating).lower() == "true":
                accuracy_count.append(0)
                false_negatives.append(1)

        
        tp = sum(true_positives)
        fp = sum(false_positives)
        fn = sum(false_negatives)
        tn = sum(true_negatives)
        accuracy = sum(accuracy_count)/len(accuracy_count)


        print("Model: ", model_name)
        print("File: ", results_file)
        print("TP", tp)
        print("FP", fp)
        print("TN", tn)
        print("FN", fn)
        print("Sum correct response format: ", sum([tp, fp, tn, fn]))

        precision = tp/(tp+fp)
        recall = tp/(tp+fn)
        f1_score = 2 * (precision * recall) / (precision + recall)

        print("Accuracy", accuracy)
        print("Precision", precision)
        print("Recall", recall)
        print("F1 Score: ", f1_score)

        results.append({
            "model_name": model_name,
            "file": results_file,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1-score": f1_score
        })
    

    df_results = pd.DataFrame(results)
    df_results.to_csv(f"{result_dir}/validation_effectiveness.csv", index=False)


compute_validation_effectiveness(
    baseline_file="../data/evaluation/all_dependencies.csv",
    model_names=["gpt-3.5-turbo-0125", "gpt-4o-2024-05-13", "llama3:70b", "llama3:8b"],
    result_dir="../data/evaluation/config2",
    index_name="all"
)

Model:  gpt-3.5-turbo-0125
File:  ../data/evaluation/config2/all_dependencies_all_gpt-3.5-turbo-0125.json
TP 150
FP 109
TN 188
FN 53
Sum correct response format:  500
Accuracy 0.676
Precision 0.5791505791505791
Recall 0.7389162561576355
F1 Score:  0.6493506493506492
Model:  gpt-4o-2024-05-13
File:  ../data/evaluation/config2/all_dependencies_all_gpt-4o-2024-05-13.json
TP 114
FP 28
TN 269
FN 89
Sum correct response format:  500
Accuracy 0.766
Precision 0.8028169014084507
Recall 0.5615763546798029
F1 Score:  0.6608695652173913
Model:  llama3:70b
File:  ../data/evaluation/config2/all_dependencies_all_llama3:70b.json
TP 156
FP 100
TN 197
FN 47
Sum correct response format:  500
Accuracy 0.706
Precision 0.609375
Recall 0.7684729064039408
F1 Score:  0.6797385620915033
'isDependency'
'isDependency'
'isDependency'
'isDependency'
'isDependency'
'isDependency'
'isDependency'
'isDependency'
'isDependency'
Model:  llama3:8b
File:  ../data/evaluation/config2/all_dependencies_all_llama3:8b.json
TP 18

In [12]:
import pandas as pd

file_path = "../data/evaluation/config2/validation_effectiveness.csv"

df = pd.read_csv(file_path)

df

,model_name,file,accuracy,precision,recall,f1-score
0,gpt-3.5-turbo-0125,../data/evaluation/config2/all_dependencies_al...,0.676000,0.579151,0.738916,0.649351
1,gpt-4o-2024-05-13,../data/evaluation/config2/all_dependencies_al...,0.766000,0.802817,0.561576,0.660870
2,llama3:70b,../data/evaluation/config2/all_dependencies_al...,0.706000,0.609375,0.768473,0.679739
3,llama3:8b,../data/evaluation/config2/all_dependencies_al...,0.464358,0.426540,0.895522,0.577849
